In [1]:
import cv2 as cv
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [3]:
mtx1 = [[1563.4010009765625*544/1920, 0.0, 986.95068359375*544/1920],
        [0.0, 1561.4229736328125*320/1080, 518.3526611328125*320/1080],
        [0.0, 0.0, 1.0]]
mtx1 = np.array(mtx1)
mtx2 = [[1564.150634765625*544/1920, 0.0, 954.2699584960938*544/1920],
        [0.0, 1562.171630859375*320/1080, 539.9055786132813*320/1080],
        [0.0, 0.0, 1.0]]
mtx2 = np.array(mtx2)

dist = [0, 0, 0, 0, 0]
dist1 = np.array(dist)
dist2 = np.array(dist)

In [5]:
def stereo_calibrate(mtx1, dist1, mtx2, dist2, frames_folder):
    #read the synched frames
    images_names = glob.glob(frames_folder)
    images_names = sorted(images_names)
    c1_images_names = images_names[:len(images_names)//2]
    c2_images_names = images_names[len(images_names)//2:]
    
    c1_images = []
    c2_images = []
    for im1, im2 in zip(c1_images_names, c2_images_names):
        _im = cv.imread(im1, 1)
        c1_images.append(_im)
 
        _im = cv.imread(im2, 1))
        c2_images.append(_im)
    
 
    #change this if stereo calibration not good.
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
 
    rows = 6 #number of checkerboard rows.
    columns = 9 #number of checkerboard columns.
    world_scaling = 24. #change this to the real world square size. Or not.
 
    #coordinates of squares in the checkerboard world space
    objp = np.zeros((rows*columns,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:columns].T.reshape(-1,2)
    objp = world_scaling* objp
 
    #frame dimensions. Frames should be the same size.
    width = c1_images[0].shape[1]
    height = c1_images[0].shape[0]
 
    #Pixel coordinates of checkerboards
    imgpoints_left = [] # 2d points in image plane.
    imgpoints_right = []
 
    #coordinates of the checkerboard in checkerboard world space.
    objpoints = [] # 3d point in real world space
 
    for frame1, frame2 in zip(c1_images, c2_images):
        gray1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
        gray2 = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        c_ret1, corners1 = cv.findChessboardCorners(gray1, (rows, columns), None)
        c_ret2, corners2 = cv.findChessboardCorners(gray2, (rows, columns), None)
        print(c_ret1, c_ret2)
        if c_ret1 == True and c_ret2 == True:
            corners1 = cv.cornerSubPix(gray1, corners1, (11, 11), (-1, -1), criteria)
            corners2 = cv.cornerSubPix(gray2, corners2, (11, 11), (-1, -1), criteria)
 
            cv.drawChessboardCorners(frame1, (rows, columns), corners1, c_ret1)
            cv.imshow('img', frame1)
 
            cv.drawChessboardCorners(frame2, (rows, columns), corners2, c_ret2)
            cv.imshow('img2', frame2)
            k = cv.waitKey(500)
 
            objpoints.append(objp)
            imgpoints_left.append(corners1)
            imgpoints_right.append(corners2)
 
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, mtx1, dist1,
                                                                 mtx2, dist2, (width, height), criteria = criteria, flags = stereocalibration_flags)
 
    print(ret)
    return R, T

In [6]:
R, T = stereo_calibrate(mtx1, dist1, mtx2, dist2, 'sy/*')
print(R)
print(T)

"R, T = stereo_calibrate(mtx1, dist1, mtx2, dist2, 'sy_table/*')\n#R_, T_ = cv.stereo_calibrate(mtx3, dist3, mtx4, dist4, 'sy/*')\nprint(R,'R_')\nprint(T,'T_')"

In [7]:
cv.destroyAllWindows()

In [8]:
T1 = T1.reshape((3,1))
T2 = T2.reshape((3,1))
print(R1)
print(R2)
print(T1)
print(T2)
RT1 = np.concatenate([R1, T1], axis = -1)
P1 = mtx1 @ RT1 #projection matrix for C1
RT2 = np.concatenate([R2, T2], axis = -1)
P2 = mtx2 @ RT2 #projection matrix for C2
print(P1)
print(P2)

[[ 0.97064433  0.1577033   0.18160189]
 [ 0.00225615 -0.76097723  0.64877467]
 [ 0.24050882 -0.62931973 -0.73899403]]
[[ 0.91076033  0.33060276  0.24742158]
 [ 0.11357348 -0.7766193   0.6196479 ]
 [ 0.39700969 -0.5362502  -0.74486175]]
[[0.36000035]
 [1.05323034]
 [0.61486616]]
[[0.70707336]
 [1.04182934]
 [0.57067286]]
[[ 4.97215054e+02 -1.06123644e+02 -1.26206324e+02  3.31405786e+02]
 [ 3.79825724e+01 -4.48716114e+02  1.86652488e+02  5.81705349e+02]
 [ 2.40508815e-01 -6.29319734e-01 -7.38994034e-01  6.14866156e-01]]
[[ 5.10969216e+02  1.52576989e+00 -9.17417941e+01  4.67654476e+02]
 [ 1.16079709e+02 -4.45255441e+02  1.67656700e+02  5.73518726e+02]
 [ 3.97009688e-01 -5.36250197e-01 -7.44861755e-01  5.70672860e-01]]


In [10]:
np.savetxt('P1.dat', P1, delimiter=',')
np.savetxt('P2.dat', P2, delimiter=',')